In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [2]:

image_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=r"C:\Users\ulzii\OneDrive\Desktop\baba\media",
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(300, 300),
    color_mode="grayscale",  
)



Found 1004 files belonging to 18 classes.


In [3]:
val_data = []
train_data = []

dataset_size = 1004
k  = 10
fold_size = 100

for i in range(k):
    val = image_data.skip(fold_size*i).take(fold_size)
    train = image_data.take(i * fold_size).concatenate(image_data.skip((i + 1) * fold_size))
    val_data.append(val)
    train_data.append(train)


In [4]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(18, activation='softmax')    
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
final_train_accuracies = []
final_val_accuracies = []

for i in range(k):
    print(f"Training Fold {i+1}...")
    train = train_data[i]
    val = val_data[i]
    
    model = create_model()
    history = model.fit(train,
              epochs=5,)
    scores = model.evaluate(val, verbose=0)

    final_train_accuracies.append(history.history['accuracy'][-1]) 
    final_val_accuracies.append(scores[-1]) 
    
    


Training Fold 1...
Epoch 1/5
904/904 [==============================] - 10s 11ms/step - loss: 48.6304 - accuracy: 0.5442  
Epoch 2/5
904/904 [==============================] - 7s 8ms/step - loss: 0.2866 - accuracy: 0.9447
Epoch 3/5
904/904 [==============================] - 7s 8ms/step - loss: 0.0539 - accuracy: 0.9834 
Epoch 4/5
904/904 [==============================] - 7s 8ms/step - loss: 0.0621 - accuracy: 0.9867 
Epoch 5/5
904/904 [==============================] - 7s 8ms/step - loss: 0.2053 - accuracy: 0.9768
Training Fold 2...
Epoch 1/5
904/904 [==============================] - 7s 8ms/step - loss: 33.1708 - accuracy: 0.5653
Epoch 2/5
904/904 [==============================] - 7s 8ms/step - loss: 0.1860 - accuracy: 0.9513
Epoch 3/5
904/904 [==============================] - 7s 8ms/step - loss: 0.0599 - accuracy: 0.9812 
Epoch 4/5
904/904 [==============================] - 7s 8ms/step - loss: 0.1481 - accuracy: 0.9712 
Epoch 5/5
904/904 [==============================] - 7s 8ms/s

In [ ]:
folds = [f'Fold {i+1}' for i in range(10)]

# Create bar positions
x = np.arange(10)  

# Set width for bars
width = 0.35

# Plotting the bar graph
plt.figure(figsize=(14, 8))

# Plotting training accuracies and validation accuracies
plt.bar(x - width/2, final_train_accuracies, width, label='Train Accuracy', color='blue')
plt.bar(x + width/2, final_val_accuracies, width, label='Val Accuracy', color='orange')

# Add labels and title
plt.xlabel('Folds')
plt.ylabel('Accuracy')
plt.title(f'{10}-Fold Cross Validation: Train vs Val Accuracy')
plt.xticks(x, folds)
plt.legend(loc='lower right')
plt.grid(True, axis='y')
plt.show()